# Test ML Models Framework

This notebook tests the multiple machine learning models framework for predicting insufficient pain management.

In [ ]:
import sys
import os
sys.path.append('/Users/jk1/icu_research/PreHosp')

from analgesia.prediction_of_insufficient_pain_management.data_preprocessing import load_and_preprocess_data
from analgesia.prediction_of_insufficient_pain_management.ml_models import MLModelEvaluator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load and preprocess the data
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/analgesia/data/trauma_categories_Rega Pain Study15.09.2025_v2.xlsx'

print("Loading and preprocessing data...")
processed_data, processor = load_and_preprocess_data(data_path)

print(f"\nProcessed data shape: {processed_data.shape}")

In [ ]:
# Prepare data for modeling
X_train, X_test, y_train, y_test = processor.prepare_modeling_data()

print(f"Training set: {X_train.shape[0]} samples, {X_train.shape[1]} features")
print(f"Test set: {X_test.shape[0]} samples, {X_test.shape[1]} features")

In [ ]:
# Initialize ML Model Evaluator
print("\nInitializing ML Model Evaluator...")
ml_evaluator = MLModelEvaluator()

# List available models
print("Available models:")
model_configs = ml_evaluator.get_model_configs()
for model_name in model_configs.keys():
    print(f"  - {model_name}")

In [ ]:
# Train and evaluate Random Forest model
print("\nTraining Random Forest model...")

# First tune the model
print("Tuning hyperparameters...")
rf_best_params = ml_evaluator.tune_model('random_forest', X_train, y_train)
print(f"Best parameters: {rf_best_params}")

# Evaluate the model
print("Evaluating model...")
rf_results = ml_evaluator.evaluate_model('random_forest', X_test, y_test)

print("\nRandom Forest Results:")
for metric, score in rf_results.items():
    if isinstance(score, (int, float)):
        print(f"  {metric}: {score:.4f}")
    elif hasattr(score, 'mean'):
        print(f"  {metric}: {score.mean():.4f} (+/- {score.std() * 2:.4f})")
    else:
        print(f"  {metric}: {score}")

In [ ]:
# Get feature importance from Random Forest
print("\nRandom Forest Feature Importance:")
rf_feature_importance = ml_evaluator.get_feature_importance('random_forest')
if rf_feature_importance is not None:
    print("Top 10 Most Important Features:")
    print(rf_feature_importance.head(10))
    
    # Check if VAS_change or VAS_improved are causing perfect predictions
    vas_features = rf_feature_importance[rf_feature_importance['feature'].str.contains('VAS', na=False)]
    if not vas_features.empty:
        print(f"\nVAS-related features importance:")
        print(vas_features)
else:
    print("Feature importance not available for this model type.")

In [ ]:
# Investigate potential data leakage from engineered features
print("Investigating potential data leakage...")

# Check correlation between VAS features and target
vas_features = ['VAS_on_scene', 'VAS_change', 'VAS_improved']
available_vas_features = [f for f in vas_features if f in X_train.columns]

print(f"\nAvailable VAS features in training set: {available_vas_features}")

if available_vas_features:
    # Reconstruct full data to check correlations
    temp_data = X_train.copy()
    temp_data['target'] = y_train
    
    for feature in available_vas_features:
        corr = temp_data[[feature, 'target']].corr().iloc[0, 1]
        print(f"Correlation between {feature} and target: {corr:.4f}")
        
        # Check unique values
        unique_vals = temp_data[feature].nunique()
        print(f"  Unique values in {feature}: {unique_vals}")
        
        # Check if feature is binary or has obvious patterns
        if unique_vals <= 10:
            value_counts = temp_data[feature].value_counts().head()
            print(f"  Value distribution: {dict(value_counts)}")
        print()

In [ ]:
# Test multiple models using run_all_models
print("Comparing multiple ML models...")

# Run all available models
print("Running all models (this may take several minutes)...")
try:
    results = ml_evaluator.run_all_models(X_train, X_test, y_train, y_test)
    
    print("\nModel Comparison Results:")
    summary = ml_evaluator.get_model_comparison_summary()
    print(summary)
    
    print("\nFinal Summary:")
    ml_evaluator.print_final_summary()
    
except Exception as e:
    print(f"Error running all models: {str(e)}")
    
    # Let's try individual models
    print("\nTrying individual models...")
    models_to_test = ['random_forest', 'gradient_boosting']
    
    for model_name in models_to_test:
        try:
            print(f"\nTesting {model_name}...")
            # Skip tuning for faster testing
            eval_results = ml_evaluator.evaluate_model(model_name, X_test, y_test)
            print(f"  Accuracy: {eval_results.get('accuracy', 'N/A'):.4f}")
            print(f"  ROC-AUC: {eval_results.get('roc_auc', 'N/A'):.4f}")
        except Exception as model_error:
            print(f"  Error with {model_name}: {str(model_error)}")